# LGBM_baseline

 DACON Baseline 코드 + LGBM 파라미터 조정만.

In [ ]:
# install lgbm
!pip install lightgbm

In [ ]:
# module import
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import lightgbm as lgb
import os
import glob

In [ ]:
# path
root_path = '/content/drive/My Drive/TOY/DACON/DACON-JEJU'
data_path = os.path.join(root_path, 'data')
submission_path = os.path.join(root_path, 'submission')

## 데이터 전처리

In [ ]:
# load data
data = pd.read_csv(os.path.join(data_path, '201901-202003.csv'))

In [ ]:
data.head()

,REG_YYMM,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,HOM_SIDO_NM,HOM_CCG_NM,AGE,SEX_CTGO_CD,FLC,CSTMR_CNT,AMT,CNT
0,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,20s,1,1,4,311200,4
1,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,1,2,7,1374500,8
2,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,2,2,6,818700,6
3,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,3,4,1717000,5
4,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,4,3,1047300,3


In [ ]:
data.isnull().sum()

REG_YYMM             0
CARD_SIDO_NM         0
CARD_CCG_NM      87213
STD_CLSS_NM          0
HOM_SIDO_NM          0
HOM_CCG_NM      147787
AGE                  0
SEX_CTGO_CD          0
FLC                  0
CSTMR_CNT            0
AMT                  0
CNT                  0
dtype: int64

In [ ]:
data['REG_YYMM'].unique()

array([201901, 201902, 201903, 201904, 201905, 201906, 201907, 201908,
       201909, 201910, 201911, 201912, 202001, 202002, 202003])

In [ ]:
# 데이터 전처리
def grap_year(data):
    data = str(data)
    return int(data[:4])

def grap_month(data):
    data = str(data)
    return int(data[4:])

In [ ]:
# 결측치 채우기
# data = data.fillna('')

# 년, 월 나누기
data['year'] = data['REG_YYMM'].apply(lambda x: grap_year(x))
data['month'] = data['REG_YYMM'].apply(lambda x: grap_month(x))
data.drop('REG_YYMM', axis=1, inplace=True)
data.head()

,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,HOM_SIDO_NM,HOM_CCG_NM,AGE,SEX_CTGO_CD,FLC,CSTMR_CNT,AMT,CNT,year,month
0,강원,강릉시,건강보조식품 소매업,강원,강릉시,20s,1,1,4,311200,4,2019,1
1,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,1,2,7,1374500,8,2019,1
2,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,2,2,6,818700,6,2019,1
3,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,3,4,1717000,5,2019,1
4,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,4,3,1047300,3,2019,1


In [ ]:
# 결측치 있는 열 삭제
df = data.drop(columns=['CARD_CCG_NM', 'HOM_CCG_NM'], axis=1)
df.head()

,CARD_SIDO_NM,STD_CLSS_NM,HOM_SIDO_NM,AGE,SEX_CTGO_CD,FLC,CSTMR_CNT,AMT,CNT,year,month
0,강원,건강보조식품 소매업,강원,20s,1,1,4,311200,4,2019,1
1,강원,건강보조식품 소매업,강원,30s,1,2,7,1374500,8,2019,1
2,강원,건강보조식품 소매업,강원,30s,2,2,6,818700,6,2019,1
3,강원,건강보조식품 소매업,강원,40s,1,3,4,1717000,5,2019,1
4,강원,건강보조식품 소매업,강원,40s,1,4,3,1047300,3,2019,1


In [ ]:
# 시도별, 업종별, 고객 지역별, 나이대별, 성별, FLC, year, month별 합으로 그룹화 : 범주형 변수별로 묶은 것.
columns = ['CARD_SIDO_NM', 'STD_CLSS_NM', 'HOM_SIDO_NM', 'AGE', 'SEX_CTGO_CD', 'FLC', 'year', 'month']
df = df.groupby(columns).sum().reset_index(drop=False)
df.head()

,CARD_SIDO_NM,STD_CLSS_NM,HOM_SIDO_NM,AGE,SEX_CTGO_CD,FLC,year,month,CSTMR_CNT,AMT,CNT
0,강원,건강보조식품 소매업,강원,20s,1,1,2019,1,4,311200,4
1,강원,건강보조식품 소매업,강원,20s,1,1,2019,2,3,605000,3
2,강원,건강보조식품 소매업,강원,20s,1,1,2019,6,3,139000,3
3,강원,건강보조식품 소매업,강원,20s,1,1,2019,8,3,27500,3
4,강원,건강보조식품 소매업,강원,20s,1,1,2019,9,3,395500,3


In [ ]:
# 열 삭제: CSTMR_CNT, CNT
df.drop(columns=['CSTMR_CNT', 'CNT'], axis=1, inplace=True)
df.head()

,CARD_SIDO_NM,STD_CLSS_NM,HOM_SIDO_NM,AGE,SEX_CTGO_CD,FLC,year,month,AMT
0,강원,건강보조식품 소매업,강원,20s,1,1,2019,1,311200
1,강원,건강보조식품 소매업,강원,20s,1,1,2019,2,605000
2,강원,건강보조식품 소매업,강원,20s,1,1,2019,6,139000
3,강원,건강보조식품 소매업,강원,20s,1,1,2019,8,27500
4,강원,건강보조식품 소매업,강원,20s,1,1,2019,9,395500


In [ ]:
# 인코딩
print(df.info())

dtypes = df.dtypes # 인덱스가 col, 값이 데이터타입.
encoders = {} # 인코딩해야 할 객체 dict로 저장.

for col in df.columns:
    if str(dtypes[col]) == 'object':
        encoder = LabelEncoder()
        encoder.fit(df[col])
        encoders[col] = encoder # 인코더 객체 저장해 놓고 나중에.

df_num = df.copy()
for col in encoders.keys():
    encoder = encoders[col]
    print(col, encoder.classes_)
    df_num[col] = encoder.transform(df[col]) # 바꾸기

print(df_num.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1057394 entries, 0 to 1057393
Data columns (total 9 columns):
 #   Column        Non-Null Count    Dtype 
---  ------        --------------    ----- 
 0   CARD_SIDO_NM  1057394 non-null  object
 1   STD_CLSS_NM   1057394 non-null  object
 2   HOM_SIDO_NM   1057394 non-null  object
 3   AGE           1057394 non-null  object
 4   SEX_CTGO_CD   1057394 non-null  int64 
 5   FLC           1057394 non-null  int64 
 6   year          1057394 non-null  int64 
 7   month         1057394 non-null  int64 
 8   AMT           1057394 non-null  int64 
dtypes: int64(5), object(4)
memory usage: 72.6+ MB
None
CARD_SIDO_NM ['강원' '경기' '경남' '경북' '광주' '대구' '대전' '부산' '서울' '세종' '울산' '인천' '전남' '전북'
 '제주' '충남' '충북']
STD_CLSS_NM ['건강보조식품 소매업' '골프장 운영업' '과실 및 채소 소매업' '관광 민예품 및 선물용품 소매업'
 '그외 기타 분류안된 오락관련 서비스업' '그외 기타 스포츠시설 운영업' '그외 기타 종합 소매업' '기타 대형 종합 소매업'
 '기타 수상오락 서비스업' '기타 외국식 음식점업' '기타 주점업' '기타음식료품위주종합소매업' '내항 여객 운송업' '마사지업'
 '면세점' '버스 운송업' '비알콜 음료점업' '빵 및

In [ ]:
display(df_num)

,CARD_SIDO_NM,STD_CLSS_NM,HOM_SIDO_NM,AGE,SEX_CTGO_CD,FLC,year,month,AMT
0,0,0,0,1,1,1,2019,1,311200
1,0,0,0,1,1,1,2019,2,605000
2,0,0,0,1,1,1,2019,6,139000
3,0,0,0,1,1,1,2019,8,27500
4,0,0,0,1,1,1,2019,9,395500
...,...,...,...,...,...,...,...,...,...
1057389,16,40,16,6,2,5,2019,3,148000
1057390,16,40,16,6,2,5,2019,5,329800
1057391,16,40,16,6,2,5,2019,10,557800
1057392,16,40,16,6,2,5,2019,12,247800


In [ ]:
# feature, target 설정
train_features = df_num.drop(columns=['AMT'], axis=1) # 훈련 데이터
train_labels = np.log1p(df_num['AMT'])

In [ ]:
# 트레인, 테스트 분리
X_train, X_test, y_train, y_test = train_test_split(train_features, train_labels, shuffle=True, test_size=0.2)

## LGBM 데이터셋

In [ ]:
# LGBM 데이터셋 구성
# train_data = lgb.Dataset(X_train, label=y_train, categorical_feature=['CARD_SIDO_NM', 'STD_CLSS_NM', 'HOM_SIDO_NM', 'AGE', 'SEX_CTGO_CD', 'FLC'])
train_data = lgb.Dataset(X_train, label=y_train)
val_data = lgb.Dataset(X_test, label=y_test)

## 파라미터 조정

In [ ]:
# 3) 6.10xx
%%time
params_ = {'boosting_type': 'gbdt',
           'objective': 'regression',
           'metric': 'rmse', 
           'learning_rate': 0.025,
           'metric_freq': 10,
           'max_depth':-1,
           'early_stopping_round': 50,
           'verbose': 1}

model = lgb.train(params=params_,
                  train_set=train_data,
                  num_boost_round=10000,
                  valid_sets=val_data,
                  verbose_eval=100,
                  early_stopping_rounds=100)

/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:123: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 50 rounds.
[100]	valid_0's rmse: 2.14827
[200]	valid_0's rmse: 1.973
[300]	valid_0's rmse: 1.87729
[400]	valid_0's rmse: 1.81124
[500]	valid_0's rmse: 1.75553
[600]	valid_0's rmse: 1.70347
[700]	valid_0's rmse: 1.65038
[800]	valid_0's rmse: 1.59635
[900]	valid_0's rmse: 1.55797
[1000]	valid_0's rmse: 1.52981
[1100]	valid_0's rmse: 1.49923
[1200]	valid_0's rmse: 1.47167
[1300]	valid_0's rmse: 1.4419
[1400]	valid_0's rmse: 1.41949
[1500]	valid_0's rmse: 1.39908
[1600]	valid_0's rmse: 1.37719
[1700]	valid_0's rmse: 1.35577
[1800]	valid_0's rmse: 1.33246
[1900]	valid_0's rmse: 1.31082
[2000]	valid_0's rmse: 1.29246
[2100]	valid_0's rmse: 1.27234
[2200]	valid_0's rmse: 1.25828
[2300]	valid_0's rmse: 1.24415
[2400]	valid_0's rmse: 1.22942
[2500]	valid_0's rmse: 1.21533
[2600]	valid_0's rmse: 1.19856
[2700]	valid_0's rmse: 1.18873
[2800]	valid_0's rmse: 1.17903
[2900]	valid_0's rmse: 1.16871
[3000]	valid_0's rmse: 1.15912
[3100]	valid_0's rms

In [ ]:
# # 2) 6.25xxxxx
# %%time
# params_ = {
#            'boosting_type': 'gbdt',
#            'objective': 'regression',
#            'metric': 'rmse', 
#            'is_training_metric': True,
#            'learning_rate': 0.025,
#            'metric_freq': 10,
#            'max_depth':-1,
#            'early_stopping_round': 50,
#            'verbose': 1}

# model = lgb.train(params=params_,
#                   train_set=train_data,
#                   num_boost_round=5000,
#                   valid_sets=val_data,
#                   verbose_eval=50,
#                   early_stopping_rounds=100,
#                  )

/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:123: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 50 rounds.
[50]	valid_0's rmse: 2.28329
[100]	valid_0's rmse: 2.14643
[150]	valid_0's rmse: 2.05096
[200]	valid_0's rmse: 1.97894
[250]	valid_0's rmse: 1.92987
[300]	valid_0's rmse: 1.8871
[350]	valid_0's rmse: 1.84599
[400]	valid_0's rmse: 1.81222
[450]	valid_0's rmse: 1.78111
[500]	valid_0's rmse: 1.75492
[550]	valid_0's rmse: 1.73333
[600]	valid_0's rmse: 1.71451
[650]	valid_0's rmse: 1.69332
[700]	valid_0's rmse: 1.67304
[750]	valid_0's rmse: 1.65122
[800]	valid_0's rmse: 1.63421
[850]	valid_0's rmse: 1.61329
[900]	valid_0's rmse: 1.58798
[950]	valid_0's rmse: 1.57262
[1000]	valid_0's rmse: 1.55504
[1050]	valid_0's rmse: 1.53947
[1100]	valid_0's rmse: 1.52427
[1150]	valid_0's rmse: 1.50693
[1200]	valid_0's rmse: 1.49182
[1250]	valid_0's rmse: 1.48042
[1300]	valid_0's rmse: 1.46553
[1350]	valid_0's rmse: 1.4541
[1400]	valid_0's rmse: 1.44293
[1450]	valid_0's rmse: 1.43272
[1500]	valid_0's rmse: 1.417
[1550]	valid_0's rmse: 1.40304
[

### 파라미터, 점수 기록용

In [ ]:
# # 1) 6.56xxxxxx
# params_ = {
#            'boosting_type': 'gbdt',
#            'objective': 'regression',
#            'metric': 'rmse', 
#            'is_training_metric': True,
#            'learning_rate': 0.025,
#            'metric_freq': 10,
#            'max_depth':-1,
#            'early_stopping_round': 50,
#            'verbose': 1}

# %%time
# model = lgb.train(params=params_,
#                   train_set=train_data,
#                   num_boost_round=2000,
#                   valid_sets=val_data,
#                   verbose_eval=50,
#                   early_stopping_rounds=100,
#                  )

/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:123: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 50 rounds.
[50]	valid_0's rmse: 2.28705
[100]	valid_0's rmse: 2.15662
[150]	valid_0's rmse: 2.05475
[200]	valid_0's rmse: 1.9864
[250]	valid_0's rmse: 1.93352
[300]	valid_0's rmse: 1.8876
[350]	valid_0's rmse: 1.8521
[400]	valid_0's rmse: 1.81719
[450]	valid_0's rmse: 1.78667
[500]	valid_0's rmse: 1.7558
[550]	valid_0's rmse: 1.72711
[600]	valid_0's rmse: 1.70381
[650]	valid_0's rmse: 1.68419
[700]	valid_0's rmse: 1.66171
[750]	valid_0's rmse: 1.64166
[800]	valid_0's rmse: 1.62288
[850]	valid_0's rmse: 1.5971
[900]	valid_0's rmse: 1.57522
[950]	valid_0's rmse: 1.55712
[1000]	valid_0's rmse: 1.54156
[1050]	valid_0's rmse: 1.52477
[1100]	valid_0's rmse: 1.50584
[1150]	valid_0's rmse: 1.4914
[1200]	valid_0's rmse: 1.48001
[1250]	valid_0's rmse: 1.46581
[1300]	valid_0's rmse: 1.45256
[1350]	valid_0's rmse: 1.44218
[1400]	valid_0's rmse: 1.43072
[1450]	valid_0's rmse: 1.41906
[1500]	valid_0's rmse: 1.40913
[1550]	valid_0's rmse: 1.4005
[160

## 예측

In [ ]:
# 예측 템플릿
CARD_SIDO_NMs = df_num['CARD_SIDO_NM'].unique()
STD_CLSS_NMs = df_num['STD_CLSS_NM'].unique()
HOM_SIDO_NMs = df_num['HOM_SIDO_NM'].unique()
AGEs = df_num['AGE'].unique()
SEX_CTGO_CDs = df_num['SEX_CTGO_CD'].unique()
FLCs = df_num['FLC'].unique()
# CANCELs = df_num['CANCEL'].unique() # 추가
years = [2020]
months = [4, 7]

temp = []
for CARD_SIDO_NM in CARD_SIDO_NMs:
    for STD_CLSS_NM in STD_CLSS_NMs:
        for HOM_SIDO_NM in HOM_SIDO_NMs:
            for AGE in AGEs:
                for SEX_CTGO_CD in SEX_CTGO_CDs:
                    for FLC in FLCs:
                        # for CANCEL in CANCELs: # 추가
                            for year in years:
                                for month in months:
                                    temp.append([CARD_SIDO_NM, STD_CLSS_NM, HOM_SIDO_NM, AGE, SEX_CTGO_CD, FLC, year, month])

temp = np.array(temp)
temp = pd.DataFrame(data=temp, columns=['CARD_SIDO_NM', 'STD_CLSS_NM', 'HOM_SIDO_NM', 'AGE', 'SEX_CTGO_CD', 'FLC', 'year', 'month'])
display(temp)

,CARD_SIDO_NM,STD_CLSS_NM,HOM_SIDO_NM,AGE,SEX_CTGO_CD,FLC,year,month
0,0,0,0,1,1,1,2020,4
1,0,0,0,1,1,1,2020,7
2,0,0,0,1,1,2,2020,4
3,0,0,0,1,1,2,2020,7
4,0,0,0,1,1,3,2020,4
...,...,...,...,...,...,...,...,...
1658855,16,30,14,0,2,3,2020,7
1658856,16,30,14,0,2,4,2020,4
1658857,16,30,14,0,2,4,2020,7
1658858,16,30,14,0,2,5,2020,4


In [ ]:
# 예측: 8개 컬럼으로 예측은 한다.
%%time
pred = model.predict(temp)
pred = np.expm1(pred)
temp['AMT'] = np.round(pred, 0)
temp['REG_YYMM'] = temp['year']*100 + temp['month']
temp_df = temp[['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM', 'AMT']]
temp_df = temp_df.groupby(['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM']).sum().reset_index(drop=False)

CPU times: user 25min 30s, sys: 460 ms, total: 25min 30s
Wall time: 6min 27s


In [ ]:
# 범주형 변수 디코딩
temp_df['CARD_SIDO_NM'] = encoders['CARD_SIDO_NM'].inverse_transform(temp_df['CARD_SIDO_NM'])
temp_df['STD_CLSS_NM'] = encoders['STD_CLSS_NM'].inverse_transform(temp_df['STD_CLSS_NM'])
display(temp_df)

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
0,202004,강원,건강보조식품 소매업,1.907080e+08
1,202004,강원,골프장 운영업,4.018921e+09
2,202004,강원,과실 및 채소 소매업,1.295784e+09
3,202004,강원,관광 민예품 및 선물용품 소매업,3.059702e+07
4,202004,강원,그외 기타 분류안된 오락관련 서비스업,4.567070e+06
...,...,...,...,...
1389,202007,충북,피자 햄버거 샌드위치 및 유사 음식점업,1.519127e+09
1390,202007,충북,한식 음식점업,3.880425e+10
1391,202007,충북,호텔업,3.235788e+07
1392,202007,충북,화장품 및 방향제 소매업,5.588105e+08


In [ ]:
# 제출 파일 순번
def return_sub_num():
    files = sorted(glob.glob(f"{submission_path}/*.csv"), key= lambda x:int(x.split('.')[0][-3:]))
    if len(files) == 0:
        return '001'
    else:
        return str(int(files[-1].split('.')[0][-3:])+1).zfill(3)

In [ ]:
num = return_sub_num()
print(num)

005


In [ ]:
# 제출파일 만들기
submission = pd.read_csv(f"{data_path}/submission.csv", index_col=0)
submission = submission.drop(columns=['AMT'], axis=1)
submission = submission.merge(temp_df, left_on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'], right_on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'], how='left')
submission.index.name = 'id'
display(submission)
submission.to_csv(f"{submission_path}/submission_{num}.csv", encoding='utf-8-sig')

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
id,,,,
0,202004,강원,건강보조식품 소매업,1.907080e+08
1,202004,강원,골프장 운영업,4.018921e+09
2,202004,강원,과실 및 채소 소매업,1.295784e+09
3,202004,강원,관광 민예품 및 선물용품 소매업,3.059702e+07
4,202004,강원,그외 기타 분류안된 오락관련 서비스업,4.567070e+06
...,...,...,...,...
1389,202007,충북,피자 햄버거 샌드위치 및 유사 음식점업,1.519127e+09
1390,202007,충북,한식 음식점업,3.880425e+10
1391,202007,충북,호텔업,3.235788e+07


In [ ]:
return_sub_num()

'006'

## rmlse 점수 체크

- `submission004` : 0.92xx
- `submission005` : 0.82xx

In [ ]:
# rmsle 점수
def rmsle(y_true, y_pred):
    assert len(y_true) == len(y_pred)
    return np.sqrt(np.mean(np.power(np.log1p(y_true + 1) - np.log1p(y_pred + 1), 2)))

In [ ]:
def check_rmsle(real_data):
    y_true = real_data['AMT']
    print(y_true)

    data_for_pred = real_data.drop(columns=['AMT'], axis=1)
    y_pred = np.expm1(model.predict(data_for_pred)) # 항상 모델으로 해놓을 것!
    print(y_pred)

    return rmsle(y_true, y_pred)

In [ ]:
%%time
print(check_rmsle(df_num))

0          311200
1          605000
2          139000
3           27500
4          395500
            ...  
1057389    148000
1057390    329800
1057391    557800
1057392    247800
1057393    230400
Name: AMT, Length: 1057394, dtype: int64
[731686.30377294 527100.54977958 242909.90861055 ... 305857.19840793
 251332.2982599  249449.0009057 ]
0.8223272757390884
CPU times: user 16min 9s, sys: 283 ms, total: 16min 9s
Wall time: 4min 6s
